# KPI Extracting and Data Cleaning 

In [1]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings('ignore')
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import math
import matplotlib.pyplot as plt 

In [3]:
# pd.set_option("display.precision", 4)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', lambda x: '%.4f' % x)
sns.set_context('talk')
sns.set_style('darkgrid', {'font.sans-serif': ['Iosevka']})

## Data Summary

In [4]:
file = 'datasets/COMP_SALE_SB_EXTRACT_20201019.csv'

In [5]:
def load_dataset(filename, na_val=None, parse_date=False, verbose=False):
    if parse_date:
        to_parse = [1,2]
        df = pd.read_csv(filename, parse_dates=to_parse, na_values=na_val)
    else:
        df = pd.read_csv(filename, na_values=na_val)
    # Features and Target column name 
    features = df.columns
    target = ''

    print("+ Dataset:")
    display(df.head(3))
    
    if verbose:
        print("+ Shape:")
        display(df.shape) 
        print(f"+ Target: {target}")
        print(f"+ Overall Variables Info: ")
        display(pd.DataFrame(df.info(memory_usage='deep')))
    
    return features, target, df

features, target, dataset = load_dataset(file, na_val=None, parse_date=False, verbose=True)

+ Dataset:


,ID,CREATED_AT,SOLD_DATE,SOLD_DURATION,ADDRESS,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,BUILDING_SIZE,BUILDING_SURFACE,LOT_SURFACE,LAND_REGISTER,BUILDING_YEAR,LOT_EVALUATION,BUILDING_EVALUATION,YEAR_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_ELECTRICITY,EXPENSES_OIL,EXPENSES_GAZ,EXPENSES_ASSURANCE,EXPENSES_SNOW,EXPENSES_MAINTENANCE,FONDATION_TYPE,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,AGENCY_ADDRESS,CT_FULL_NAME,CT_EMAIL,DISTANCE
0,101,2020-05-29 08:24:07,2018-12-31,381,431-433 12e Avenue N. Sherbrooke (Fleurimont) ...,45.450564,-71.860709,125000,2,2,29.9 X 35.2 ft / 9.07 X 10.72 m,"1,047.6 sqft / 97.33 sqm","6,124.66 sqft / 569 sqm",1 329 778 Cadastre du Quebec,1952,71100,98400,2014,15600,0,0,0,2386,267,0,0,0,0,0,0,Concrete slab on ground,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,157 boul. Jacques-Cartier S. Sherbrooke J1J 2Z4,Katy Rheaume,katyrheaume@remax-quebec.com,9.056856
1,102,2020-05-29 08:24:13,2019-01-15,323,118 5e Avenue Sherbrooke (Fleurimont) J1G 2L5,45.403658,-71.877727,369000,2,0,37 X 44 ft irr / 11.28 X 13.41 m irr,"1,630 sqft / 151.43 sqm","6,476 sqft / 601.64 sqm",1132557,1949,75100,206200,2018,1,0,0,0,3476,803,4694,0,0,0,0,0,Poured concrete,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,25 rue Bocage Suite 102 Sherbrooke J1L 2J4,Patrick Fares,patrick.fares@century21.ca,4.891268
2,103,2020-05-29 08:24:19,2019-01-23,86,502 Rue King E. Sherbrooke (Fleurimont) J1G 1B5,45.407861,-71.877697,340000,6,3,30.1 X 41.1 ft / 9.16 X 12.52 m,"1,234.4 sqft / 114.68 sqm","4,500.39 sqft / 418.1 sqm",1330580,1972,104500,225300,2018,28452,0,0,0,5164,562,371,0,0,1426,500,0,Poured concrete,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,30 Rioux Sherbrooke J1J 2W7,Dominic Glaude,dominic.glaude@gmail.com,5.067245


+ Shape:


(1500, 46)

+ Target: 
+ Overall Variables Info: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    1500 non-null   int64  
 1   CREATED_AT            1500 non-null   object 
 2   SOLD_DATE             1500 non-null   object 
 3   SOLD_DURATION         1500 non-null   int64  
 4   ADDRESS               1500 non-null   object 
 5   LATITUDE              1500 non-null   float64
 6   LONGITUDE             1500 non-null   float64
 7   SOLD_PRICE            1500 non-null   int64  
 8   UNIT_NUMBER           1500 non-null   int64  
 9   PROPERTY_FLOOR        1500 non-null   int64  
 10  BUILDING_SIZE         1400 non-null   object 
 11  BUILDING_SURFACE      1103 non-null   object 
 12  LOT_SURFACE           1462 non-null   object 
 13  LAND_REGISTER         1500 non-null   object 
 14  BUILDING_YEAR         1500 non-nul

""


In [6]:
def display_missing(dataset, verbose=False):
    features = dataset.columns
    if verbose:
        print("+ Missing data: ")
    missing_count = dataset[features].isnull().sum()
    missing_percent = (dataset[features].isnull().sum() / len(dataset) * 100).round(4)
    missing = pd.DataFrame({'feature': features, 'count': missing_count, 'percent': missing_percent,})
    missing.sort_values('count', ascending=False, inplace=True)
    missing.reset_index(drop=True, inplace=True)
    return missing

display_missing(dataset, True)

+ Missing data: 


,feature,count,percent
0,BUILDING_SURFACE,397,26.4667
1,FONDATION_TYPE,209,13.9333
2,HEATING_ENERGY,172,11.4667
3,BUILDING_SIZE,100,6.6667
4,LOT_SURFACE,38,2.5333
5,NUMBER_5_H,0,0.0000
6,EXPENSES_ASSURANCE,0,0.0000
7,EXPENSES_SNOW,0,0.0000
8,EXPENSES_MAINTENANCE,0,0.0000
9,NUMBER_1_H,0,0.0000


## 1. KPI Extracting 

In [7]:
# KPI Extractor functions
def get_rba(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    return row["RESIDENTIAL_REVENUE"] + row["COMMERCIAL_REVENUE"] + row["PARKING_REVENUE"] + row["OTHERS_REVENUE"]

def get_round_up(row):
    if np.isnan(row["RBA"]):
        return np.nan
    decimals = -3
    multiplier = 10 ** decimals
    return math.ceil(row["RBA"] * multiplier) / multiplier

def get_revenue_gross_round(row):
    if np.isnan(row["RBA"]):
        return np.nan
    return round(row["RBA"], -3)

def get_norm_expense_vac(row):
    if np.isnan(row["RBA"]):
        return np.nan
    return round(row["RBA"] * 0.03, 2)

def get_norm_expense_ins(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if np.isnan(row["EXPENSES_ASSURANCE"]) or row["EXPENSES_ASSURANCE"] < 100:
        if row["UNIT_NUMBER"] > 5:
            return row["UNIT_NUMBER"] * 500
        else:
            return row["UNIT_NUMBER"] * 650
    else:
        return row["EXPENSES_ASSURANCE"]

def get_norm_expense_maint(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if row["UNIT_NUMBER"] >= 5:
        return row["UNIT_NUMBER"] * 500
    else:
        return np.nan

def get_norm_expense_conc(row):
    if np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if 4 <= row["UNIT_NUMBER"] <= 6:
        row = row["UNIT_NUMBER"] * 125
    elif 7 <= row["UNIT_NUMBER"] <= 11:
        row = row["UNIT_NUMBER"] * 170
    else:
        row = row["UNIT_NUMBER"] * 300
    return row

def get_norm_expense_mngt(row):
    if np.isnan(row["RBA"]) or np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if 5 <= row["UNIT_NUMBER"] <= 6:
        row = row["RBA"] * .03
    elif 7 <= row["UNIT_NUMBER"] <= 11:
        row = row["RBA"] * .04
    else:
        row = row["RBA"] * .05
    return row

def get_norm_expense_elec(row):
    if np.isnan(row["EXPENSES_ELECTRICITY"]):
        return np.nan
    else:
        return row["EXPENSES_ELECTRICITY"]

def get_norm_expense_heat(row):
    if np.isnan(row["EXPENSES_OIL"]) or np.isnan(row["EXPENSES_GAZ"]):
        if not np.isnan(row["EXPENSES_OIL"]):
            return row["EXPENSES_OIL"]
        if not np.isnan(row["EXPENSES_GAZ"]):
            return row["EXPENSES_GAZ"]
        else:
            return np.nan
    return row["EXPENSES_OIL"] + row["EXPENSES_GAZ"]

def get_total_norm_expenses(row):
    if np.isnan(row["UNIT_NUMBER"]) or np.isnan(row["RBA"]):
        return np.nan
    total_exp = 0
    if row["UNIT_NUMBER"] >= 5:
        total_exp = \
            row["MUNICIPAL_TAXE"] + row["SCHOOL_TAXE"] + \
            row["NORM_EXP_MAINTENANCE"] + row["NORM_EXP_CONCIERGE"] + \
            row["NORM_EXP_MANAGEMENT"] + row["NORM_EXP_INSURANCE"] + \
            row["NORM_EXP_ELECTRICITY"] + row["NORM_EXP_HEAT"] + \
            row["NORM_EXP_VACANCY"]

    if 2 <= row["UNIT_NUMBER"] <= 4 and row["RBA"] > 5000:
        total_exp = \
            row["MUNICIPAL_TAXE"] + row["SCHOOL_TAXE"] + \
            row["EXPENSES_ASSURANCE"] + row["NORM_EXP_INSURANCE"] + \
            row["NORM_EXP_ELECTRICITY"] + row["NORM_EXP_HEAT"] + \
            row["NORM_EXP_VACANCY"] + row["EXPENSES_SNOW"]
    return total_exp

def get_total_norm_expenses_rounded(row):
    if np.isnan(row["NORM_TOTAL_EXPENSES"]):
        return np.nan
    return abs(round(row["NORM_TOTAL_EXPENSES"], -2))

def get_net_norm_income(row):
    if np.isnan(row["RBA"]) or np.isnan(row["NORM_TOTAL_EXPENSES"]):
        return np.nan
    return round(row["RBA"] - row["NORM_TOTAL_EXPENSES"], 2)

def get_cap_rate(row):
    if np.isnan(row["NET_NORM_INCOME"]) or np.isnan(row["SOLD_PRICE"]):
        return np.nan
    return round((row["NET_NORM_INCOME"] / row["SOLD_PRICE"] * 100), 2)

def get_gross_revenue_mult(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["RBA"]) or row["RBA"] == 0:
        return np.nan
    return round(row["SOLD_PRICE"] / row["RBA"], 2)

def get_unit_price(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["UNIT_NUMBER"]):
        return np.nan
    if row["UNIT_NUMBER"] == 0:
        return row["UNIT_NUMBER"]
    return round(row["SOLD_PRICE"] / row["UNIT_NUMBER"], 2)

def get_net_revenue_mult(row):
    if np.isnan(row["SOLD_PRICE"]) or np.isnan(row["NET_NORM_INCOME"]):
        return np.nan
    if row["NET_NORM_INCOME"] == 0:
        return row["SOLD_PRICE"]
    return round(row["SOLD_PRICE"] / row["NET_NORM_INCOME"], 2)

In [8]:
df = dataset.copy()
df["RBA"] = df.apply(get_rba, axis=1)
df["REVENUE_GROSS_ROUND"] = df.apply(get_revenue_gross_round, axis=1)
df["NORM_EXP_HEAT"] = df.apply(get_norm_expense_heat, axis=1)
df["NORM_EXP_VACANCY"] = df.apply(get_norm_expense_vac, axis=1)
df["NORM_EXP_INSURANCE"] = df.apply(get_norm_expense_ins, axis=1)
df["NORM_EXP_MAINTENANCE"] = df.apply(get_norm_expense_maint, axis=1)
df["NORM_EXP_CONCIERGE"] = df.apply(get_norm_expense_conc, axis=1)
df["NORM_EXP_MANAGEMENT"] = df.apply(get_norm_expense_mngt, axis=1)
df["NORM_EXP_ELECTRICITY"] = df.apply(get_norm_expense_elec, axis=1)
df["NORM_TOTAL_EXPENSES"] = df.apply(get_total_norm_expenses, axis=1)
df["TOTAL_NORM_EXP_ROUNDED"] = df.apply(get_total_norm_expenses_rounded, axis=1)
df["NET_NORM_INCOME"] = df.apply(get_net_norm_income, axis=1)
df["CAP_RATE"] = df.apply(get_cap_rate, axis=1)
df["GROSS_REVENUE_MULTIPLICATOR"] = df.apply(get_gross_revenue_mult, axis=1)
df["UNIT_PRICE"] = df.apply(get_unit_price, axis=1)
df["NET_REVENUE_MULTIPLICATOR"] = df.apply(get_net_revenue_mult, axis=1)

df.to_csv(r'datasets/COMP_SALE_SB_EXTRACT_KPI_20201020.csv', index=False)

In [9]:
# Cross check the data 
file = 'datasets/COMP_SALE_SB_EXTRACT_KPI_20201020.csv' 
features, target, dataset = load_dataset(file, na_val=None, parse_date=False)


+ Dataset:


,ID,CREATED_AT,SOLD_DATE,SOLD_DURATION,ADDRESS,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,BUILDING_SIZE,BUILDING_SURFACE,LOT_SURFACE,LAND_REGISTER,BUILDING_YEAR,LOT_EVALUATION,BUILDING_EVALUATION,YEAR_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_ELECTRICITY,EXPENSES_OIL,EXPENSES_GAZ,EXPENSES_ASSURANCE,EXPENSES_SNOW,EXPENSES_MAINTENANCE,FONDATION_TYPE,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,AGENCY_ADDRESS,CT_FULL_NAME,CT_EMAIL,DISTANCE,RBA,REVENUE_GROSS_ROUND,NORM_EXP_HEAT,NORM_EXP_VACANCY,NORM_EXP_INSURANCE,NORM_EXP_MAINTENANCE,NORM_EXP_CONCIERGE,NORM_EXP_MANAGEMENT,NORM_EXP_ELECTRICITY,NORM_TOTAL_EXPENSES,TOTAL_NORM_EXP_ROUNDED,NET_NORM_INCOME,CAP_RATE,GROSS_REVENUE_MULTIPLICATOR,UNIT_PRICE,NET_REVENUE_MULTIPLICATOR
0,101,2020-05-29 08:24:07,2018-12-31,381,431-433 12e Avenue N. Sherbrooke (Fleurimont) ...,45.450564,-71.860709,125000,2,2,29.9 X 35.2 ft / 9.07 X 10.72 m,"1,047.6 sqft / 97.33 sqm","6,124.66 sqft / 569 sqm",1 329 778 Cadastre du Quebec,1952,71100,98400,2014,15600,0,0,0,2386,267,0,0,0,0,0,0,Concrete slab on ground,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,157 boul. Jacques-Cartier S. Sherbrooke J1J 2Z4,Katy Rheaume,katyrheaume@remax-quebec.com,9.056856,15600,16000,0,468.00,1300,NaN,600,780.00,0,4421.00,4400.0,11179.00,8.94,8.01,62500.00,11.18
1,102,2020-05-29 08:24:13,2019-01-15,323,118 5e Avenue Sherbrooke (Fleurimont) J1G 2L5,45.403658,-71.877727,369000,2,0,37 X 44 ft irr / 11.28 X 13.41 m irr,"1,630 sqft / 151.43 sqm","6,476 sqft / 601.64 sqm",1132557,1949,75100,206200,2018,1,0,0,0,3476,803,4694,0,0,0,0,0,Poured concrete,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,25 rue Bocage Suite 102 Sherbrooke J1L 2J4,Patrick Fares,patrick.fares@century21.ca,4.891268,1,0,0,0.03,1300,NaN,600,0.05,4694,0.00,0.0,1.00,0.00,369000.00,184500.00,369000.00
2,103,2020-05-29 08:24:19,2019-01-23,86,502 Rue King E. Sherbrooke (Fleurimont) J1G 1B5,45.407861,-71.877697,340000,6,3,30.1 X 41.1 ft / 9.16 X 12.52 m,"1,234.4 sqft / 114.68 sqm","4,500.39 sqft / 418.1 sqm",1330580,1972,104500,225300,2018,28452,0,0,0,5164,562,371,0,0,1426,500,0,Poured concrete,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,30 Rioux Sherbrooke J1J 2W7,Dominic Glaude,dominic.glaude@gmail.com,5.067245,28452,28000,0,853.56,1426,3000.0,750,853.56,371,12980.12,13000.0,15471.88,4.55,11.95,56666.67,21.98


## 2. Data Cleanse 

In [10]:
def get_bldg_surface(row):
    if pd.isna(row["BUILDING_SURFACE"]):
        return np.nan
    return float(row["BUILDING_SURFACE"].split()[0].replace(',', ''))

def get_lot_surface(row):
    if pd.isna(row["LOT_SURFACE"]):
        return np.nan
    return float(row["LOT_SURFACE"].split()[0].replace(',', ''))

def get_fondation_quality(row):
    if pd.isna(row["FONDATION_TYPE"]):
        return np.nan
    elif "Other" in row["FONDATION_TYPE"].strip():
        return 1
    elif ("Stone" or \
            "Stone, Other" or \
            "Concrete blocks, Stone, Other" or \
            "Poured concrete, Stone, Other" or \
            "Concrete blocks, Stone") in row["FONDATION_TYPE"].strip():
        return 3
    elif ("Concrete slab on ground, Poured concrete, Stone" or \
            "Concrete slab on ground, Stone" or \
            "Concrete") in row["FONDATION_TYPE"].strip():
        return 4
    elif ("Poured concrete, Stone" or \
            "Concrete blocks, Poured concrete, Stone") in row["FONDATION_TYPE"].strip():
        return 5
    elif ("Concrete slab on ground" or \
            "Concrete blocks" or \
            "Concrete slab on ground, Poured concrete" or \
            "Concrete blocks, Poured concrete, Other" or \
            "Concrete blocks, Concrete slab on ground" or \
            "Concrete slab on ground, Other" or \
            "Concrete blocks, Other") in \
          row["FONDATION_TYPE"].strip():
        return 6
    elif ("Poured concrete, Other" or "Concrete") in row["FONDATION_TYPE"].strip():
        return 7
    elif ("Poured concrete" or "Concrete blocks, Poured concrete") in row["FONDATION_TYPE"].strip():
        return 8
    else:
        return np.nan
    


In [11]:
# extract sizes only 
df = dataset.copy()
df["BUILDING_SURFACE_SQFT"] = df.apply(get_bldg_surface, axis=1)
df["LOT_SURFACE_SQFT"] = df.apply(get_lot_surface, axis=1)
df["FONDATION_QUAL"] = df.apply(get_fondation_quality, axis=1)


In [12]:
display_missing(df, True)

+ Missing data: 


,feature,count,percent
0,NORM_EXP_MAINTENANCE,1158,77.2000
1,BUILDING_SURFACE_SQFT,397,26.4667
2,BUILDING_SURFACE,397,26.4667
3,FONDATION_QUAL,227,15.1333
4,FONDATION_TYPE,209,13.9333
5,HEATING_ENERGY,172,11.4667
6,BUILDING_SIZE,100,6.6667
7,LOT_SURFACE,38,2.5333
8,LOT_SURFACE_SQFT,38,2.5333
9,GROSS_REVENUE_MULTIPLICATOR,9,0.6000


In [13]:
# Remove redundant/unecessary features 
cols_drop = ["ID", "CREATED_AT", "SOLD_DATE", "ADDRESS", "RBA",
            "BUILDING_SIZE", "BUILDING_SURFACE", "LOT_SURFACE", "LAND_REGISTER", 
            "FONDATION_TYPE", "AGENCY_ADDRESS", "CT_FULL_NAME", "CT_EMAIL", 
            "EXPENSES_MAINTENANCE", "EXPENSES_ELECTRICITY", "EXPENSES_OIL", "EXPENSES_GAZ",
            "EXPENSES_ASSURANCE",  "NORM_EXP_MAINTENANCE", "NORM_TOTAL_EXPENSES"]


# Save dataset having "Salary Estimate"
df.drop(cols_drop, inplace=True, axis=1)
# Save 
df.to_csv('datasets/COMP_SALE_SB_CLEAN_20201104.csv', index=False)

In [14]:
# Cross check the data 
file = 'datasets/COMP_SALE_SB_CLEAN_20201104.csv' 
features, target, dataset = load_dataset(file, na_val=None, parse_date=False)

+ Dataset:


,SOLD_DURATION,LATITUDE,LONGITUDE,SOLD_PRICE,UNIT_NUMBER,PROPERTY_FLOOR,BUILDING_YEAR,LOT_EVALUATION,BUILDING_EVALUATION,YEAR_EVALUATION,RESIDENTIAL_REVENUE,COMMERCIAL_REVENUE,PARKING_REVENUE,OTHERS_REVENUE,MUNICIPAL_TAXE,SCHOOL_TAXE,EXPENSES_SNOW,HEATING_ENERGY,NUMBER_1_H,NUMBER_2_H,NUMBER_3_H,NUMBER_4_H,NUMBER_5_H,NUMBER_6_H,NUMBER_7_H,NUMBER_8_H,NUMBER_9_H,AGENCY_NAME,DISTANCE,REVENUE_GROSS_ROUND,NORM_EXP_HEAT,NORM_EXP_VACANCY,NORM_EXP_INSURANCE,NORM_EXP_CONCIERGE,NORM_EXP_MANAGEMENT,NORM_EXP_ELECTRICITY,TOTAL_NORM_EXP_ROUNDED,NET_NORM_INCOME,CAP_RATE,GROSS_REVENUE_MULTIPLICATOR,UNIT_PRICE,NET_REVENUE_MULTIPLICATOR,BUILDING_SURFACE_SQFT,LOT_SURFACE_SQFT,FONDATION_QUAL
0,381,45.450564,-71.860709,125000,2,2,1952,71100,98400,2014,15600,0,0,0,2386,267,0,"Electricity, Other",0,0,0,0,2,0,0,0,0,RE/MAX D'ABORD INC.,9.056856,16000,0,468.00,1300,600,780.00,0,4400.0,11179.00,8.94,8.01,62500.00,11.18,1047.6,6124.66,6.0
1,323,45.403658,-71.877727,369000,2,0,1949,75100,206200,2018,1,0,0,0,3476,803,0,"Dual energy, Electricity, Heating oil",0,0,0,0,0,0,1,0,0,CENTURY 21 ACTIONWEB INC.,4.891268,0,0,0.03,1300,600,0.05,4694,0.0,1.00,0.00,369000.00,184500.00,369000.00,1630.0,6476.00,8.0
2,86,45.407861,-71.877697,340000,6,3,1972,104500,225300,2018,28452,0,0,0,5164,562,500,Electricity,0,0,6,0,0,0,0,0,0,Via Capitale Estrie,5.067245,28000,0,853.56,1426,750,853.56,371,13000.0,15471.88,4.55,11.95,56666.67,21.98,1234.4,4500.39,8.0


In [15]:
display_missing(df, True)

+ Missing data: 


,feature,count,percent
0,BUILDING_SURFACE_SQFT,397,26.4667
1,FONDATION_QUAL,227,15.1333
2,HEATING_ENERGY,172,11.4667
3,LOT_SURFACE_SQFT,38,2.5333
4,GROSS_REVENUE_MULTIPLICATOR,9,0.6000
5,NORM_EXP_INSURANCE,0,0.0000
6,NUMBER_8_H,0,0.0000
7,NUMBER_9_H,0,0.0000
8,AGENCY_NAME,0,0.0000
9,DISTANCE,0,0.0000
